# Geospatial Visualisation

In [26]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as ctx


import sys, os
sys.path.append(os.path.abspath('../'))
from scripts.utils import create_dir, get_runtime
import time
start_time = time.time()

In [ ]:
# load sa2 shapefile
sa2_gdf = gpd.read_file('../data/landing/sa2/sa2-21-shp/')
sa2_gdf['sa2_code'] = sa2_gdf['SA2_CODE21']
sa2_gdf = sa2_gdf[['sa2_code', 'geometry']]
sa2_gdf

In [ ]:
# load rental history/current dataset 2017 - 2024
rental_df = pd.read_csv('../data/curated/rental-17-24.csv')
rental_df = rental_df[
    ['sa2_code', 'suburb', 'bed', 'bath', 'car', 'type', 'year', 'median_income', 'population', 'time_city', 'avg_property_price', 'rented_price']
]
rental_df['sa2_code'] = rental_df['sa2_code'].astype('Int64').astype(str)
rental_df = rental_df.merge(sa2_gdf, on='sa2_code',how='left')

rental_df

In [21]:
house_df = rental_df[rental_df['type'] == 'House']
unit_df = rental_df[rental_df['type'] == 'Unit/apmt']
print('House:', house_df.shape)
print('Unit:', unit_df.shape)

In [ ]:
house_agg_df = house_df.groupby(['sa2_code', 'geometry']).agg({
    'median_income': 'mean',
    'population': 'mean',
    'time_city': 'mean',
    'avg_property_price': 'mean',
    'rented_price': 'mean',
}).reset_index()
house_agg_df

In [ ]:
unit_agg_df = unit_df.groupby(['sa2_code', 'geometry']).agg({
    'median_income': 'mean',
    'population': 'mean',
    'time_city': 'mean',
    'avg_property_price': 'mean',
    'rented_price': 'mean',
}).reset_index()
unit_agg_df

In [51]:
# Function to plot heatmaps for different columns
def plot_geospatial_heatmap_osm_save(gdf, column, title, cmap, property_type):
    """ Plot a heatmap of a given column in a GeoDataFrame with OpenStreetMap as background.
    """    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plot the Victoria SA2 boundaries with heatmap and a custom color scheme
    gdf.plot(column=column, ax=ax, legend=True, cmap=cmap, alpha=0.7,
             legend_kwds={'shrink': 0.5, 'label': f"{title} (Scale)"})  # Adjust legend size and label
    
    # Add OpenStreetMap as background
    ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)
    
    # Title and labels
    ax.set_title(f'{property_type} {title}', fontsize=18, fontweight='bold', pad=20)
    ax.set_axis_off()
    
    # Adjust layout and save the plot
    plt.tight_layout()
    plt.savefig(f"../plots/{property_type.lower()}_{column}.png", dpi=300)  # Save the figure with high resolution
    plt.show()

# List of columns and corresponding color maps to plot
columns_to_plot = {
    'median_income': 'OrRd',            # Orange-Red color scheme for income
    'population': 'Blues',              # Blue color scheme for population
    'avg_property_price': 'Greens',     # Green color scheme for property price
    'rented_price': 'Purples'           # Purple color scheme for rented price
}

In [ ]:
# Plot heatmaps for house data
house_agg_gdf = gpd.GeoDataFrame(house_agg_df, crs='EPSG:4326', geometry=house_agg_df['geometry'])
for column, cmap in columns_to_plot.items():
    plot_geospatial_heatmap_osm_save(house_agg_gdf, column, column.replace('_', ' ').title(), cmap, 'House')

In [ ]:
# Plot heatmaps for unit data
unit_agg_gdf = gpd.GeoDataFrame(unit_agg_df, crs='EPSG:4326', geometry=unit_agg_df['geometry'])
for column, cmap in columns_to_plot.items():
    plot_geospatial_heatmap_osm_save(unit_agg_gdf, column, column.replace('_', ' ').title(), cmap, 'Unit_Apt')